In [149]:
import os
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from collections import OrderedDict
import time

import re


def get_ipo_data(url):
    info = OrderedDict()

    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup('table')

    #### 공모정보 테이블에서 데이터 가져오기 ####
    info_table = list(filter(lambda x: x.attrs.get(
        'summary') == '공모정보', tables))[0]  # 공모정보
    info_tds = info_table.select('td')
    info_texts = [i.text for i in info_tds]

    # 0~9가 아닌 것을 ''으로 교체, 총 공모주식수
    info['ipo_total'] = re.sub('[^0-9]', '', info_texts[1])
    info['face_value'] = re.sub('[^0-9]', '', info_texts[3])  # 액면가

    ## 신주,구주발행
    ipo_rates = re.findall('\((.*?)\)', info_texts[5])
    if len(ipo_rates) == 1:
        info['new_rate'] = ipo_rates[0]
        info['old_rate'] = None
    else:
        info['new_rate'] = ipo_rates[0]
        info['old_rate'] = ipo_rates[1]

    info['ipo_type'] = info_texts[5]  # 공모타입
    info['wish_price_min'] = re.sub(
        '[^0-9]', '', info_texts[7].split(" ~ ")[0])  # 희망공모가(최저)
    info['wish_price_max'] = re.sub(
        '[^0-9]', '', info_texts[7].split(" ~ ")[1])  # 희망공모가(최고)
    info['sub_rate'] = info_texts[9]  # 청약경쟁률
    info['final_price'] = re.sub('[^0-9]', '', info_texts[11])
    info['host_party'] = info_texts[15]

    #### 청약일정의 공모사항 테이블에서 데이터 가져오기 ####
    # x is bs element

    details_table = list(filter(lambda x: x.attrs.get('summary') == '공모청약일정', tables))[
        0].select_one('table')  # 공모정보
    # details_table = list(filter(lambda x: "청약증거금율" in x.text, tables))[1]  ## 공모사항
    details_trs = details_table.select('tr')
    info['employee_rate'] = details_trs[5].select('td')[2].text  # 비중
    info['employee_margin'] = details_trs[5].select(
        'td')[3].text.replace("청약증거금율 : ", "")

    info['org_rate'] = details_trs[6].select('td')[1].text
    info['normal_rate'] = details_trs[7].select('td')[1].text
    info['normal_margin'] = details_trs[7].select(
        'td')[2].text.replace("청약증거금율 : ", "")

    def clean_dict(input_dict):  # my_dict is a list of dict
        ini_list = [
            '15일 확약',
            '1개월 확약',
            '3개월 확약',
            '6개월 확약',
            '총 수량 대비 비율(%)',
        ]
        template_dict = OrderedDict(zip(ini_list, [None]*len(ini_list)))

        for key, value in input_dict.items():
            template_dict[key] = value
        return template_dict
    ############# 의무보율확약비율 ##############
    try:
        mand_table = list(
            filter(lambda x: "총 수량 대비 비율(%)" in x.text, tables))[-1]  # 공모사항
        mand_table_trs = mand_table.select('tr')
        ## "15일 확약"
        ## "1개월 확약"
        ## "3개월 확약"
        ## "6개월 확약"
        # "합계"
        ## "15일 확약"

        # 어차피 0은 header, -2은 합계, -1은 총 수량 대비 비율



        tmp_dict = OrderedDict()
        for i, d in enumerate(mand_table_trs[1:]):
            tmp_dict[d.select('td')[0].text] = d.select('td')[1].text
        tmp_dict = clean_dict(tmp_dict)

        info.update(tmp_dict)
    except:
        tmp_dict = OrderedDict()
        tmp_dict = clean_dict(tmp_dict)
        info.update(tmp_dict)

    ############ 수요예측 참여내역
    try:
        demand_part_table = list(
            filter(lambda x: "단순경쟁" in x.text, tables))[-1]  # 수요예측
        info['demand_count'] = demand_part_table.select(
            'tr')[1].select('td')[0].text
        info['demand_ratio'] =  demand_part_table.select(
            'tr')[1].select('td')[2].text.replace(":1","")
    except:
        info['demand_count'] = None
        info['demand_ratio'] = None

    return info



def getIPOElements(page_no):
    url = 'https://www.38.co.kr/html/fund/index.htm?o=nw&page={}'.format(page_no)
    # url로부터 신규상장종목 리스트가 들어있는 table 태그를 가져온다. (38커뮤니케이션)
    # summary 속성이 '신규상장종목'인 태그 솔팅

    html = requests.get(url.format(page_no)).text
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup('table')
    table_ipo_list = list(filter(lambda x: x.attrs.get('summary') == '신규상장종목', tables))[0]

    # 테이블 헤더 태그(thead)를 읽는다.
    head = table_ipo_list('thead')[0]
    th = head('tr')[0]('th')
    column_headers = [x.text for x in th]

    # 테이블 본체 (tbody)를 읽어서 리스트에 딕셔너리 형태로 저장
    result = []
    tbody = table_ipo_list('tbody')[0]
    rows = tbody('tr')
    for row in rows:
        td = row('td')
        info = OrderedDict()
        for i, e in enumerate(td):
            header_text = column_headers[i]

            if len(header_text) == 0:
                continue
            temp = e.text.replace('\xa0', '')
            temp = temp.replace('%', '')

            if header_text in ['공모가(원)', '시초가(원)', '첫날종가(원)', '현재가(원)']:
                if temp in ['-', '예정', '상장']:
                    info[header_text] = '' # NaN
                else:
                    info[header_text] = int(temp.replace(',', ''))
            elif header_text in ['공모가대비등락률(%)', '시초/공모(%)', '전일비(%)']:
                if temp in ['-'] or len(temp) == 0:
                    info[header_text] = ''
                else:
                    info[header_text] = float(temp)
            elif header_text in ['신규상장일']:
                info[header_text] = datetime.strptime(e.text, '%Y/%m/%d')
            elif header_text in ['기업명']:
                info[header_text] = temp.replace('(유가)', '')
                target_no = e.select_one('a')['href'][1:]
                print(f'https://www.38.co.kr/html/fund/{target_no}')
                info['url'] = f'https://www.38.co.kr/html/fund{target_no}'
            else:
                info[header_text] = e.text
        info.update(get_ipo_data(info['url']))
        result.append(info)

    return result

# a = getIPOElements(1)
# for i in a:
#     print(i)

# 해당 연도부터 현재까지 상장한 IPO 종목 pandas DF화(스팩, 리츠 제외)
def getIPOAllList(year: int) -> pd.DataFrame:
    result = list()
    page_no = 1 # 1페이지부터 탐색

    while True:
        time.sleep(0.3)
        page_result = getIPOElements(page_no)
        result.extend(page_result)

        dates = [x.get('신규상장일') for x in page_result]
        find = list(filter(lambda x: x.year < year, dates))
        if len(find) > 0:
            break
        page_no += 1
    # 스팩, 리츠 종목은 제외
    result = list(filter(lambda x: x.get('신규상장일').year >= year and '스팩' not in x.get('기업명') and '리츠' not in x.get('기업명'), result))

    return pd.DataFrame(result)

a = getIPOAllList(2017)
a.to_csv('2017_v3.csv')

https://www.38.co.kr/html/fund//?o=v&no=1799&l=
https://www.38.co.kr/html/fund//?o=v&no=1797&l=
https://www.38.co.kr/html/fund//?o=v&no=1792&l=
https://www.38.co.kr/html/fund//?o=v&no=1795&l=
https://www.38.co.kr/html/fund//?o=v&no=1794&l=
https://www.38.co.kr/html/fund//?o=v&no=1791&l=
https://www.38.co.kr/html/fund//?o=v&no=1793&l=
https://www.38.co.kr/html/fund//?o=v&no=1788&l=
https://www.38.co.kr/html/fund//?o=v&no=1790&l=
https://www.38.co.kr/html/fund//?o=v&no=1789&l=
https://www.38.co.kr/html/fund//?o=v&no=1787&l=
https://www.38.co.kr/html/fund//?o=v&no=1783&l=
https://www.38.co.kr/html/fund//?o=v&no=1784&l=
https://www.38.co.kr/html/fund//?o=v&no=1786&l=
https://www.38.co.kr/html/fund//?o=v&no=1785&l=
https://www.38.co.kr/html/fund//?o=v&no=1780&l=
https://www.38.co.kr/html/fund//?o=v&no=1779&l=
https://www.38.co.kr/html/fund//?o=v&no=1782&l=
https://www.38.co.kr/html/fund//?o=v&no=1781&l=
https://www.38.co.kr/html/fund//?o=v&no=1778&l=
https://www.38.co.kr/html/fund//?o=v&no=

In [ ]:
t3 = "  신주모집 : 4,500,000 주 (96.77%) 
	/   구주매출 : 150,000 주 (3.23%)    """

In [145]:
test_url = "https://www.38.co.kr/html/fund/?o=v&no=1737&l=" ##lg엔솔
test2 = "https://www.38.co.kr/html/fund//?o=v&no=1799&l="
print(get_ipo_data(test2))


OrderedDict([('ipo_total', '1180000'), ('face_value', '500'), ('new_rate', '12.29%'), ('ipo_type', '\xa0 신주모집 : 1,035,000 주 (87.71%)\xa0\n\t/ \xa0 구주매출 : 145,000 주 (12.29%)    '), ('wish_price_min', '34000'), ('wish_price_max', '44000'), ('sub_rate', '\xa0  '), ('final_price', ''), ('host_party', '\xa0 삼성증권\n'), ('employee_rate', '\xa0  주 \xa0      \xa0 '), ('employee_margin', '\xa0100% '), ('org_rate', '\xa0 826,000~885,000 주 \xa0(70~75%)      \xa0 '), ('normal_rate', '\xa0 295,000~354,000 주 \xa0(25~30%)      \xa0 '), ('normal_margin', '\xa050% '), ('15일 확약', None), ('1개월 확약', None), ('3개월 확약', None), ('6개월 확약', None), ('총 수량 대비 비율(%)', None), ('demand_count', ''), ('demand_ratio', '')])


In [99]:
def clean_dict(input_dict):  ## my_dict is a list of dict
    ini_list= [
    '15일 확약',
    '1개월 확약',
    '3개월 확약',
    '6개월 확약',
    '총 수량 대비 비율(%)',
    ]
    template_dict = OrderedDict(zip(ini_list, [None]*len(ini_list)))
    
    for key, value in input_dict.items():
        template_dict[key] = value
    return template_dict

t_dict = {'15일 확약':'12312321'}
print(clean_dict(t_dict))


OrderedDict([('15일 확약', '12312321'), ('1개월 확약', None), ('3개월 확약', None), ('6개월 확약', None), ('총 수량 대비 비율(%)', None)])


In [27]:
t2 = """
총공모주식수 
  42,500,000 주
 액면가 
  500 원


상장공모
  신주모집 : 34,000,000 주 (80%) 
	/   구주매출 : 8,500,000 주 (20%)    


희망공모가액
   257,000 ~ 300,000 원
 청약경쟁률 
"""

print("청약경쟁률" in t2)

True


In [20]:
import re
t = ' \xa0 257,000 ~ 300,000 원'
wish_price_min = re.sub('[^0-9]','', t.split(" ~ ")[0])
wish_price_max = re.sub('[^0-9]','', t.split(" ~ ")[1])
print(wish_price_max, wish_price_min)

300000 257000
